In [ ]:
#set initial weights     
model.load_weights('/home/mengjiao/model_weights/initial_model_svm.npy')

#set checkpoints to save weights and start training
checkpoint = ModelCheckpoint('/home/mengjiao/weights/best_model_svm'+str(nb_repeat)+str(nb_cv)+'.npy', verbose=1, monitor='val_accuracy', mode='auto', save_best_only=True)
callbacks_list = [checkpoint]
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=0, callbacks=callbacks_list)

In [ ]:
#set best weights
weights_file='/home/mengjiao/weights/best_model_svm'+str(nb_repeat)+str(nb_cv)+'.npy'
model.load_weights(weights_file)

#print model evaluate score
score=model.evaluate(X_test,Y_test)
print(score)

#obtain predict_prob, pred and confusion matrix
#predict output
predictions = model.predict(X_test)
y_pred = (predictions > 0.5).argmax(axis=1)
y_pred_proba=predictions[:, 1]
y_true =Y_test.argmax(axis=1)

#confusion matrix
matrix = metrics.confusion_matrix(y_true, y_pred)     

#print accuracy, sensitivity and specificity
total = sum(sum(matrix))
accuracy = (matrix[0, 0] + matrix[1, 1]) / total
specificity=matrix[0, 0] / (matrix[0, 0] + matrix[0, 1])
sensitivity=matrix[1, 1] / (matrix[1, 0] + matrix[1, 1]) 
  
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

#plot ROC curve
y_true=np.array(y_true).flatten()
y_pred=np.array(y_pred).flatten()
auc = roc_auc_score(y_true,y_pred)
print('AUC: %.3f' % auc)

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_true,y_pred)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()